In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Model, GPT2Config, PreTrainedModel
from datasets import load_dataset, Dataset, DatasetDict

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

import math

In [2]:
class GPT2Trader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        # use levine 2020 layer numbers
        n_layer = round((math.log(config.n_embd) - 5.039) / 5.55e-2)
        n_layer = max(1, n_layer)
        print(f'Using {n_layer} layers')
        config.n_layer = n_layer
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.norm = nn.LayerNorm(config.n_embd)
        self.gpt = GPT2Model(config)
        self.copy = nn.Linear(config.n_embd, config.n_embd, bias = False)
        self.regress = nn.Linear(config.n_embd, 3, bias = False)
        self.classify = nn.Linear(config.n_embd, 2, bias = False)
        self.trade = nn.Linear(config.n_embd, 11, bias = False)
        
        self.MSELoss = nn.MSELoss()
        self.CrossEntropyLoss = nn.CrossEntropyLoss()
        
        self.trade_sign = nn.Parameter(torch.Tensor([1, -1]), requires_grad = False)
        
    def forward(self, ohlcv, future):
        embed = self.norm(self.embed(ohlcv))
        hidden = self.gpt(inputs_embeds = embed).last_hidden_state
        
#         copy = self.copy(hidden).reshape(-1, hidden.shape[-1])
#         regression_pred = self.regress(hidden).reshape(-1, 3)
        classification = self.classify(hidden).reshape(-1, 2)
        soft_trade = self.trade(hidden)
        
        # information
        original_close = ohlcv.select(index = 3, dim = -1).unsqueeze(-1)
        future_close = future.select(index = 2, dim = -1).unsqueeze(-1)
        delta = (original_close - future_close).reshape(-1)
        class_sign = (delta >= 0).long()
        
        # sharpe information
        soft_trade = F.tanh(soft_trade) / 11
        soft_trade = soft_trade.sum(dim = -1).reshape(-1)
        soft_trade = soft_trade * delta
        
#         copy_loss = self.MSELoss(copy, embed.reshape(-1, hidden.shape[-1]).detach())
#         mse_loss = self.MSELoss(regression_pred, future.reshape(-1, 3)) / 1e-3
#         ce_loss = self.CrossEntropyLoss(classification, class_sign)
        sharpe_loss = torch.exp(-soft_trade / 1e-4).mean()
        
        loss = sharpe_loss # + ce_loss
        
#         print(f"sharpe {sharpe.mean().item():.2},  sharpe loss {sharpe_loss.item():.3}, ce_loss {ce_loss.item():.4}, mse_loss {mse_loss.item():.4}, copy_loss {copy_loss.item():.4}")
        print(f"sharpe {soft_trade.sum().item():.2}, sharpe loss {sharpe_loss.item():.3}")
#         print(f"preds {regression_pred[0].detach(), future.reshape(-1, 3)[0].detach()}")
        
        return {"loss": loss}

In [10]:
class TraderTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training :class:`~torch.utils.data.DataLoader`.

        Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
        to distributed training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            shuffle=False, # IMPORTANT TO STOP OVERFITTING
#             sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

In [11]:
eurusd = Dataset.from_parquet('data/EURUSD_day.pq', )
eurusd = DatasetDict({
    'train': Dataset.from_dict(eurusd[:3000]),
    'validation': Dataset.from_dict(eurusd[3000:3200]),
    'test': Dataset.from_dict(eurusd[3200:])
})

Using custom data configuration default-c57c77d507616140
Found cached dataset parquet (C:/Users/micha/.cache/huggingface/datasets/parquet/default-c57c77d507616140/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [12]:
def step(self, closure = None):
    """
    Performs a single optimization step.
    Arguments:
        closure (`Callable`, *optional*): A closure that reevaluates the model and returns the loss.
    """
    loss = None
    if closure is not None:
        loss = closure()

    for group in self.param_groups:
        for p in group["params"]:
            if p.grad is None:
                continue
#             grad = -p.grad.data # SEE NEGATIVE SIGN FOR MAXIMIZATION
            grad = p.grad.data
            if grad.is_sparse:
                raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

            state = self.state[p]

            # State initialization
            if len(state) == 0:
                state["step"] = 0
                # Exponential moving average of gradient values
                state["exp_avg"] = torch.zeros_like(p.data)
                # Exponential moving average of squared gradient values
                state["exp_avg_sq"] = torch.zeros_like(p.data)

            exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
            beta1, beta2 = group["betas"]

            state["step"] += 1

            # Decay the first and second moment running average coefficient
            # In-place operations to update the averages at the same time
            exp_avg.mul_(beta1).add_(grad, alpha=(1.0 - beta1))
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1.0 - beta2)
            denom = exp_avg_sq.sqrt().add_(group["eps"])

            step_size = group["lr"]
            if group["correct_bias"]:  # No bias correction for Bert
                bias_correction1 = 1.0 - beta1 ** state["step"]
                bias_correction2 = 1.0 - beta2 ** state["step"]
                step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

            p.data.addcdiv_(exp_avg, denom, value=-step_size)

            # Just adding the square of the weights to the loss function is *not*
            # the correct way of using L2 regularization/weight decay with Adam,
            # since that will interact with the m and v parameters in strange ways.
            #
            # Instead we want to decay the weights in a manner that doesn't interact
            # with the m/v parameters. This is equivalent to adding the square
            # of the weights to the loss with plain (non-momentum) SGD.
            # Add weight decay at the end (fixed version)
            if group["weight_decay"] > 0.0:
                p.data.add_(p.data, alpha=(-group["lr"] * group["weight_decay"]))

    return loss

from transformers import AdamW
AdamW.step = step

In [13]:
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 50,
    eval_steps = 50,
    report_to = "none",
    learning_rate = 1e-4,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    max_grad_norm = 1,
)

PyTorch: setting up devices


In [14]:
config = GPT2Config(
    n_embd = 384, n_head = 6, vocab_size = 0, n_positions = 2000,
    resid_pdrop = .1, embd_pdrop = .1, attn_pdrop = .1,
#     resid_pdrop = .0, embd_pdrop = .0, attn_pdrop = .0,
    scale_attn_by_inverse_layer_idx = True, use_cache = False
)

In [15]:
model = GPT2Trader(config).cuda()
trainer = TraderTrainer(
    model = model,
    args = training_args,
    train_dataset = eurusd['train'],
    eval_dataset = eurusd['validation']
)

Using 16 layers


In [16]:
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3000
Could not estimate the number of tokens of the input, floating-point operations will not be computed


sharpe -0.46, sharpe loss 2.49e+28


Step,Training Loss,Validation Loss
50,498796748543897957323243520.000000,1.009478
100,9.687800,2.189955
150,3.288600,1.093245
200,1.196200,1.024876
250,1.206200,1.000496
300,1.054200,1.007517
350,1.253300,1.011395
400,1.032300,1.036274
450,1.095600,1.016221
500,1.097900,1.140761


sharpe -0.16, sharpe loss 1.58e+09
sharpe 0.2, sharpe loss 6.53e+08
sharpe 0.08, sharpe loss 2.02e+05
sharpe -0.16, sharpe loss 1.88e+10
sharpe 0.013, sharpe loss 17.9
sharpe 0.14, sharpe loss 9.32
sharpe 0.012, sharpe loss 2.04e+03
sharpe 0.063, sharpe loss 2.26e+07
sharpe -0.3, sharpe loss 7.98e+04
sharpe 0.34, sharpe loss 23.5
sharpe 0.17, sharpe loss 3.52
sharpe -0.038, sharpe loss 7.26e+04
sharpe -0.021, sharpe loss 74.7
sharpe -0.056, sharpe loss 7.27e+04
sharpe 0.28, sharpe loss 2.52e+04
sharpe -0.027, sharpe loss 3.61e+03
sharpe -0.019, sharpe loss 8.09e+06
sharpe -0.073, sharpe loss 1.92e+02
sharpe 0.04, sharpe loss 1.42
sharpe -0.032, sharpe loss 14.2
sharpe -0.12, sharpe loss 2.57e+02
sharpe 0.076, sharpe loss 2.79
sharpe 0.0078, sharpe loss 1.58
sharpe 0.06, sharpe loss 1.15
sharpe 0.01, sharpe loss 7.47
sharpe -0.053, sharpe loss 4.64e+02
sharpe 0.0012, sharpe loss 2.33
sharpe 0.0027, sharpe loss 1.33
sharpe 0.011, sharpe loss 1.24
sharpe -0.053, sharpe loss 4.96e+08
sharp

***** Running Evaluation *****
  Num examples = 200
  Batch size = 1


...

sharpe -0.0014, sharpe loss 1.01
sharpe -0.00023, sharpe loss 1.0
sharpe 0.00031, sharpe loss 0.998
sharpe -0.00054, sharpe loss 1.0
sharpe 0.00057, sharpe loss 0.997
sharpe -0.0011, sharpe loss 1.01
sharpe -0.0004, sharpe loss 1.0
sharpe -0.00087, sharpe loss 1.01
sharpe 0.00029, sharpe loss 0.998
sharpe -0.0023, sharpe loss 1.02
sharpe -0.00081, sharpe loss 1.01
sharpe -0.00031, sharpe loss 1.0
sharpe 0.00048, sharpe loss 0.997
sharpe -0.00013, sharpe loss 1.0
sharpe 0.00057, sharpe loss 0.996
sharpe -0.00081, sharpe loss 1.01
sharpe -0.0005, sharpe loss 1.0
sharpe 2.6e-06, sharpe loss 1.0
sharpe -0.00097, sharpe loss 1.01
sharpe 0.00033, sharpe loss 0.998
sharpe -0.00069, sharpe loss 1.01
sharpe 0.001, sharpe loss 0.993
sharpe -0.00065, sharpe loss 1.0
sharpe -0.00046, sharpe loss 1.0
sharpe -0.00099, sharpe loss 1.01
sharpe 0.00063, sharpe loss 0.996
sharpe -0.00025, sharpe loss 1.0
sharpe -0.00038, sharpe loss 1.0
sharpe -0.00071, sharpe loss 1.01
sharpe 0.0013, sharpe loss 0.991


***** Running Evaluation *****
  Num examples = 200
  Batch size = 1


sharpe -0.0014, sharpe loss 1.01
sharpe -0.00022, sharpe loss 1.0
sharpe 0.00031, sharpe loss 0.998
sharpe -0.00053, sharpe loss 1.0
sharpe 0.00056, sharpe loss 0.997
sharpe -0.0011, sharpe loss 1.01
sharpe -0.00039, sharpe loss 1.0
sharpe -0.00085, sharpe loss 1.01
sharpe 0.00029, sharpe loss 0.998
sharpe -0.0023, sharpe loss 1.02
sharpe -0.00079, sharpe loss 1.01
sharpe -0.0003, sharpe loss 1.0
sharpe 0.00047, sharpe loss 0.997
sharpe -0.00013, sharpe loss 1.0
sharpe 0.00055, sharpe loss 0.996
sharpe -0.00079, sharpe loss 1.01
sharpe -0.00048, sharpe loss 1.0
sharpe 2.4e-06, sharpe loss 1.0
sharpe -0.00095, sharpe loss 1.01
sharpe 0.00032, sharpe loss 0.998
sharpe -0.00067, sharpe loss 1.01
sharpe 0.001, sharpe loss 0.993
sharpe -0.00063, sharpe loss 1.0
sharpe -0.00045, sharpe loss 1.0
sharpe -0.00097, sharpe loss 1.01
sharpe 0.00061, sharpe loss 0.996
sharpe -0.00024, sharpe loss 1.0
sharpe -0.00037, sharpe loss 1.0
sharpe -0.00069, sharpe loss 1.01
sharpe 0.0012, sharpe loss 0.992

Saving model checkpoint to ./results\checkpoint-3000
Configuration saved in ./results\checkpoint-3000\config.json
Model weights saved in ./results\checkpoint-3000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3000, training_loss=8.313279142398299e+24, metrics={'train_runtime': 2008.8731, 'train_samples_per_second': 1.493, 'train_steps_per_second': 1.493, 'total_flos': 0.0, 'train_loss': 8.313279142398299e+24, 'epoch': 1.0})

In [17]:
del trainer
del model
torch.cuda.empty_cache()

# Appendix

## quick timing check

In [ ]:
model = GPT2Trader(config).cuda()

In [72]:
%%timeit
fake_data = torch.randn(4, 391, 256)
fake_data = fake_data.cuda()
model(fake_data)
cpu = fake_data.cpu()

28.2 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
model = GPT2Trader(config).cpu()

Using 9 layers


In [74]:
%%timeit
fake_data = torch.randn(4, 391, 256)
model(fake_data)

748 ms ± 82.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
